In [1]:
import cv2
from PIL import Image
from backbones.ghost_model import GhostNet
import torch
import os
import numpy as np
from tqdm import tqdm
from face_alignment import align

from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
# import keras_cv_attention_models
import GhostFaceNets, GhostFaceNets_with_Bias
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from matlab_cp2tform import get_similarity_transform_for_cv2


2024-07-23 12:29:24.550467: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 12:29:24.578364: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 12:29:24.776397: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 12:29:24.777163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 12:29:26.628181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

model = tf.keras.models.load_model('./models/GN_W1.3_S2_ArcFace_epoch48.h5', compile=False)
model.compile()


2024-07-23 12:29:32.016743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 12:29:32.017078: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']

In [4]:
African_wom = os.listdir('../images/test/data/African/Woman')
Caucasian_wom = os.listdir('../images/test/data/Caucasian/Woman')
Asian_wom = os.listdir('../images/test/data/Asian/Woman')
Indian_wom = os.listdir('../images/test/data/Indian/Woman')
Asian_man = os.listdir('../images/test/data/Asian/Man')
African_man = os.listdir('../images/test/data/African/Man')
Caucasian_man = os.listdir('../images/test/data/Caucasian/Man')
Indian_man = os.listdir('../images/test/data/Indian/Man')

In [5]:
def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img/ 255.) - 0.5) / 0.5
    tensor = tf.convert_to_tensor(np.array([brg_img.transpose(2,0,1)]))    
    return tensor

In [6]:
cache = {}

In [7]:
def alignment(src_img,src_pts):
    ref_pts = [ [30.2946, 51.6963],[65.5318, 51.5014],
        [48.0252, 71.7366],[33.5493, 92.3655],[62.7299, 92.2041] ]
    crop_size = (96, 112)
    src_pts = np.array(src_pts).reshape(5,2)

    s = np.array(src_pts).astype(np.float32)
    r = np.array(ref_pts).astype(np.float32)

    tfm = get_similarity_transform_for_cv2(s, r)
    face_img = cv2.warpAffine(src_img, tfm, crop_size)
    return face_img


In [8]:
for race in races:
    landmark = {}
    with open(f'../images/test/txts/{race}/{race}_lmk.txt') as f:
        landmark_lines = f.readlines()
    for line in landmark_lines:
        l = line.strip().split()
        landmark[l[0]] = [float(k) for k in l[2:]]
    with open(f"../images/test/txts/{race}/{race}_pairs.txt", 'r') as f:
        pairs = f.readlines()
    pairs = [pair.strip().split() for pair in pairs]
    for pair in tqdm(pairs):
        try:
            race2 = None
            gen2 = None
            if pair[0] in eval(f'{race}_wom'):
                    gender = "Woman"
            else:
                    gender = "Man"
            sim = 0
            img1 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[1]}.jpg"
            if len(pair) == 3:
                img2 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[2]}.jpg"
                sim = 1
                name1 = pair[0]+'/'+pair[0]+'_'+'{:04}.jpg'.format(int(pair[1]))
                name2 = pair[0]+'/'+pair[0]+'_'+'{:04}.jpg'.format(int(pair[2]))
            else:
                if pair[2] in African_wom or pair[2] in Caucasian_wom or pair[2] in Asian_wom or pair[2] in Indian_wom:
                    gen2 = "Woman"
                    if pair[2] in African_wom:
                        race2 = "African"
                    elif pair[2] in Caucasian_wom:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_wom:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                else:
                    gen2 = "Man"
                    if pair[2] in African_man:
                        race2 = "African"
                    elif pair[2] in Caucasian_man:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_man:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                img2 = f"../images/test/data/Just_Images/{race2}/{gen2}/{pair[2]}_000{pair[3]}.jpg"
                name1 = pair[0]+'/'+pair[0]+'_'+'{:04}.jpg'.format(int(pair[1]))
                name2 = pair[2]+'/'+pair[2]+'_'+'{:04}.jpg'.format(int(pair[3]))
            if img1 in cache:
                 vec1 = cache[img1]
            else:
                with torch.no_grad():
                    vec1 = model(np.expand_dims(np.array(cv2.resize(alignment(cv2.imread(img1),landmark[f"/test/data/{race}/{name1}"]),(112,112))),axis=0))
                vec1/=np.linalg.norm(vec1)
                cache[img1] = vec1
            if img2 in cache:
                vec2 = cache[img2]
            else:
                with torch.no_grad():
                    vec2 = model(np.expand_dims(np.array(cv2.resize(alignment(cv2.imread(img2),landmark[f"/test/data/{race}/{name2}"]),(112,112))),axis=0))        
                vec2/=np.linalg.norm(vec2)
                cache[img2] = vec2
            if img1 not in cache:
                    cache[img1] = vec1
            if img2 not in cache:
                    cache[img2] = vec2
            score = np.dot(vec1, vec2.T)[0][0]
            with open(f"./lmks/{race}_lmks.csv", 'a') as f:
                f.write(f"{img1},{img2},{score},{sim}\n")
        except Exception as e:
            print(pair, e)
            continue

  0%|          | 0/6000 [00:00<?, ?it/s]/home/dysl-ai/Desktop/Results-on-RFW/GhostFace/matlab_cp2tform.py:50: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)
 25%|██▌       | 1520/6000 [03:14<07:23, 10.11it/s]

['m.02r7h1g', '2', 'm.0bk56n', '5'] OpenCV(4.10.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:2825: error: (-215:Assertion failed) src.cols > 0 && src.rows > 0 in function 'warpAffine'



100%|██████████| 6000/6000 [11:17<00:00,  8.85it/s]
